In [1]:
import numpy as np
import pandas as pd
from   sklearn.compose         import *
from   sklearn.experimental    import enable_iterative_imputer
from   sklearn.impute          import *
from   sklearn.linear_model    import LinearRegression 
from   sklearn.linear_model    import LogisticRegression, RidgeClassifier
from   sklearn.metrics         import mean_absolute_error
from   sklearn.model_selection import train_test_split
from   sklearn.pipeline        import Pipeline
from   sklearn.preprocessing   import *
from   sklearn.metrics         import balanced_accuracy_score
from   sklearn.inspection      import permutation_importance
from   sklearn.decomposition   import PCA
from   sklearn.dummy           import DummyClassifier
from   sklearn.ensemble        import RandomForestClassifier
from   sklearn.model_selection import RandomizedSearchCV
from   sklearn.base            import BaseEstimator, TransformerMixin

In [2]:
df = pd.read_csv('data/sub_COVID-19_Case_Surveillance_Public_Use_Data.csv')

### Remove rows with missing or unknown value in target (death_yn)
Assumption of Supervised Machine Learning is that each instance has a label.  We will discard instances with missing/unknown targets before beginning
(Target transformations are ok to do outside of pipeline)

In [3]:
df_clean_target = df[(df['death_yn'] != 'Unknown') & (df['death_yn'] != 'Missing')]

### Separate target from rest of DataFrame
Split into Train, Validation, and Test sets

In [4]:
df_X = df_clean_target.drop('death_yn', axis=1)
df_y = pd.DataFrame(df_clean_target['death_yn'])
X = df_X.to_numpy()
y = df_y.to_numpy()

In [5]:
X_train_pre, X_test, y_train_pre, y_test = train_test_split(X, y, train_size=0.8)
X_train, X_validate, y_train, y_validate = train_test_split(X_train_pre, y_train_pre, train_size=0.8)

### Build Preprocessing Pipeline
Our dataset has lots of missing values, some of which have different indicators.  i.e. some are np.nan, others are strings "Missing" or "Unknown"

In [6]:
# get categorical columns
categorical_columns = (df_X.dtypes == object)

In [7]:
# continuous variable preprocessing pipeline
con_pipe = Pipeline([('imputer', SimpleImputer(missing_values=np.nan, strategy='median', add_indicator=True)),
                     ('scaler', StandardScaler())
                    ])

# categorical variable preprocessing pipeline
cat_pipe = Pipeline([('imputer_nan', SimpleImputer(missing_values=np.nan, strategy='most_frequent', add_indicator=True)),
                     ('imputer_missing', SimpleImputer(missing_values='Missing', strategy='most_frequent', add_indicator=True)),
                     ('imputer_unknown', SimpleImputer(missing_values='Unknown', strategy='most_frequent', add_indicator=True)),
                     ('ohe'    , OneHotEncoder(handle_unknown='ignore'))
                    ])

# combine preprocessing together
preprocessing = ColumnTransformer([('categorical', cat_pipe, categorical_columns),
                                   ('continuous' , con_pipe, ~categorical_columns)
                                  ])

### Simple attempt to get baseline BA score
Just getting a starting Balanced Accuracy score so that I can tell if future adjustments are making tangible improvements or not.  Fitting a Logistic Regression model with no hyperparameters (aside from the solver, as the default threw errors), we get a 'starting' score of roughly 0.66.

I chose *Balanced Accuracy score* here as this is a binary classification problem with a significant imbalance in the target column, both of which are problems that BA socre handles well.

In [8]:
pipe = Pipeline([('prep' , preprocessing),
                 ('lg' , LogisticRegression(solver='liblinear'))
                ])

In [9]:
pipe.fit(X_train, y_train.ravel())
y_pred = pipe.predict(X_validate)
balanced_accuracy_score(y_validate.ravel(), y_pred)

0.664964360904198

### Hyperparameter Tuning and Cross Validation
Printing available hyperparameters for tuning with Cross Validation.  Good for seeing which type each hyperparameters accepts, as well as just noting all available hyperparameters.

In [10]:
RandomForestClassifier().get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [11]:
LogisticRegression().get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [12]:
RidgeClassifier().get_params()

{'alpha': 1.0,
 'class_weight': None,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'normalize': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.001}

### Construct dummy Pipeline
In cross validation, the Dummy Estimator will be replaced with the specified algorithms in the search space.  The preprocessing pipeline is built above, and is applied first (mostly just handling missing values here).

In [13]:
class DummyEstimator(BaseEstimator):
    "Pass through class, methods are present but do nothing."
    def fit(self): pass
    def score(self): pass

pipe = Pipeline([
                 ('prep', preprocessing),
                 ('clf', DummyEstimator())
                ])

### Define Search Space for Hyperparameter tuning
Cross Validation will try different each estimator algorithm with different subsets of the specified hyperparameter values specified in the search space.  Here I just give the CV a fairly wide breadth of options to choose from just to ensure we get as close to the best model possible as we can.

Values for C and penalty were given to Logistic Regression to see if forms of regularization would help.
Ridge classifier's max_iter was also given a wide range starting at 1 to see if early stopping would prove beneficial.
Random forest was given a range for max_depth of its trees, max_features per split, bootstrapping(T/F), criterion, and n_estimators, the best combination of which would be chosen by the cross validation

The most notable feature in the search space here is 'class_weight', which is available for all algorithms.  I've set the cross validation to choos between None and 'balanced' for this, although I would be very surprised if None was chosen over 'balanced', considering the target column has a ton of 'no' values and very few 'yes' values (target imbalance)

In [14]:
search_space = [
                 # LogisticRegression
                 {'clf' : [LogisticRegression(solver='liblinear')],
                  'clf__penalty': ['l1', 'l2'],
                  'clf__C' : np.logspace(0, 4, 10),
                  'clf__class_weight' : [None, 'balanced']
                 },
    
                 # RidgeClassifier
                 {'clf' : [RidgeClassifier(solver='auto')],
                  'clf__tol' : [0.001, 0.01, 0.1],
                  'clf__max_iter' : [None, 1, 10, 100, 1000],
                  'clf__class_weight' : [None, 'balanced'],
                  'clf__normalize' : [False, True]
                 },
                 
                 # RandomForest
                 {'clf' : [RandomForestClassifier(n_jobs=-1)],
                  'clf__criterion' : ['gini', 'entropy'],
                  'clf__class_weight' : [None, 'balanced'],
                  'clf__max_depth' : list(range(2,11)),
                  'clf__max_features' : ['auto', 'log2', 'sqrt'],
                  'clf__n_estimators' : list(range(50, 250, 50)),
                  'clf__bootstrap' : [True, False]
                 }
    
    
               ]

### Apply Random CV
RandomCV is similar to the standard GridSearchCV.  RandomCV is significantly faster and will still generally yield an extremely good set of hyperparamters as its result.

In [15]:
clf_algos_rand = RandomizedSearchCV(estimator=pipe,
                                    param_distributions=search_space,
                                    scoring='balanced_accuracy',
                                    n_iter=100,
                                    cv=5,
                                    n_jobs=-1)

### Fit and get best model
#### NOTE: This takes a while to run.  Feel free to skip this step in your execution, as it is not necessary to run the remainder of code.
#### Takes significantly longer on Google Colab

In [16]:
# Run CrossValidation and print out the chosen set of hyperparameters that maximized Balanced Accuracy Score
best_model = clf_algos_rand.fit(X_train, y_train.ravel())
best_model.best_estimator_

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputer_nan',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='most_frequent')),
                                                                  ('imputer_missing',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 missing_values='Missing',
                                                                                 strategy='most_frequent')),
                                                                  ('imputer_unknown',
                                                                   SimpleImputer(add_indicator=True,
                            

### Final Hyperparameters
I ran the RandomCV a number of times to verify (due to the random nature). RandomForestClassifier with class_weight='balanced' was by far the most significant contributor to the higher balanced accuracy score.

In [17]:
clf_pipe = Pipeline([(
                       'clf', RandomForestClassifier(class_weight='balanced',
                                                     max_depth=10,
                                                     max_features='sqrt',
                                                     bootstrap=False,
                                                     n_estimators = 150,
                                                     n_jobs=-1,
                                                     criterion='entropy')
)])

In [18]:
# Final pipeline (for train against validation)
pipe = Pipeline([
                 ('prep' , preprocessing),
                 ('rf', clf_pipe )
                ])


### Fit model, get Balanced Accuracy Score
Getting the new Balanced Accuracy score to compare the improvement over the original LogisticRegression model.  Note that the original score was around 0.66, while the below usually yields around 0.90

Also note that this is fit on the train set, and evaluated against the validation set.  Our final model will be fit on train + validation, and evaluated against the test set.

In [31]:
pipe.fit(X_train, y_train.ravel())
y_pred = pipe.predict(X_validate)
bal = (balanced_accuracy_score(y_validate.ravel(), y_pred))
bal

0.9001436450224278

### Fit against test set for final evaluation
For our final model, we want to combine train and validation to train our model on, as this will give the model the maximum amount of data to train on.  We then compare to the test set, which must only be opened once.

As we can see, the balanced accuracy score on the test set is also roughly 0.90.

In [32]:
# final end-to-end pipeline (for train+valid against test set)
pipe = Pipeline([
                 ('prep' , preprocessing),
                 ('rf', clf_pipe )
                ])


# X_train_pre and y_train_pre are the train sets before being split into train and validation.
# i.e. they are already the combined sets of their respective train and validation split.
pipe.fit(X_train_pre, y_train_pre.ravel())
y_pred = pipe.predict(X_test)
bal = (balanced_accuracy_score(y_test.ravel(), y_pred))
bal

0.9000255754475703

### Results
Through use of Random Forest Classifiers and automated hyperparameter search, our testing balanced accuracy comes out to roughly 90%, a 24% improvement of our initial Logistic Regression model.